In [1]:
import numpy as np
import pandas as pd

In [ ]:
#keywords, rating by other people

In [ ]:
#game section - assert if its not in a list
# more weight to recent choices
#automate choices per hour: time_now - time_of_choice
#does not take into account game section: relevant parameters are (%games of this section/all relevant games) and (%choosing of this section)
# interval in which we determine whether they played
#open/doesnt open video
#% of the video the customer watched- if they reached to the end the weight goes up faster
#tell the therapist if the kid tried their games and what are their opinions
#keywords-based grading
#LLM-based grading
#other people's feedback - what do people think? does it fit to the target?
#returns to the game
# rate 1-5
#most effective sequence


In [13]:
#Define kid
kid_profile = {"Name": "Itay", "Age": 8, "Social experties": "leadership", "self experties": "decision making", \
               "emotional experties": "laughter"}
specifics = "birthday, kids, challenging+"

kid_profile

{'Name': 'Itay',
 'Age': 8,
 'Social experties': 'leadership',
 'self experties': 'decision making',
 'emotional experties': 'laughter'}

In [201]:
#Define games
games = pd.DataFrame (columns= ["Name", "Word count", 'Game section', "number of sections in description", \
                                 "Clip length [sec]"])
games.loc [len (games), :] = ["embassador's speech", 37, "Popular", 3, 22]
games.loc [len (games), :] = ["living letters", 12, "other", 4, 10]
games.loc [len (games), :] = ["press interview", 20, "Therapist's recommendation", 1, 50]
games.loc [len (games), :] = ["common dancing", 11, "other", 1, 10]
games["choices_at_15:00"] = games["choices_at_18:00"] = games ["Total choices"] = games ["Your rating"] = 0
games


,Name,Word count,Game section,number of sections in description,Clip length [sec],choices_at_15:00,choices_at_18:00,Total choices,Your rating
0,embassador's speech,37,Popular,3,22,0,0,0,0
1,living letters,12,other,4,10,0,0,0,0
2,press interview,20,Therapist's recommendation,1,50,0,0,0,0
3,common dancing,11,other,1,10,0,0,0,0


In [202]:
chosen_games_log = pd.DataFrame ({"game played": ["living letters", "living letters", "press interview", "common dancing", \
                    "common dancing", "embassador's speech", "living letters", "press interview", \
                    "common dancing", "embassador's speech", "living letters", "press interview", \
                    "living letters", "common dancing", "press interview", "press interview"], 
                    "Time": [1800, 1800, 1500, 1500, 1800, 1500, 1800, 1500, 1500, 1500, 1800, 1800, 1500, 1800, 1500, 1800],
                    "Free text": ["boring", "fun", "amazing", "boring", "fun", "fun", "boring", "fun", \
                                  "fun", "bad", "ok", "blablabla", "boring", "fun", "bad", "bad"]})
chosen_games_log

,game played,Time,Free text
0,living letters,1800,boring
1,living letters,1800,fun
2,press interview,1500,amazing
3,common dancing,1500,boring
4,common dancing,1800,fun
5,embassador's speech,1500,fun
6,living letters,1800,boring
7,press interview,1500,fun
8,common dancing,1500,fun
9,embassador's speech,1500,bad


In [203]:
#take choices into account
for key, val in chosen_games_log.groupby ("game played"):
    game_index = list (games ["Name"]).index (key)
    games.loc [game_index, "Total choices"] += 1

    for ignore, choice in val.iterrows(): #choice is every row in val, which is every time the user chose this specific game (key)
        #Hour
        if choice ["Time"] == 1500:
            games.loc [game_index, "choices_at_15:00"] += 1
        else:
            games.loc [game_index, "choices_at_18:00"] += 1

        #Your rating
        m2_keywords = ["bad"] #minus 2
        m1_keywords = ["boring"] #minus 1
        p1_keywords = ["fun"] #plus 1
        p2_keywords = ["amazing"] #plus 2
        free_text = choice ["Free text"]
        total_rating = 0
        if free_text in (m2_keywords):
            total_rating -= 2
        elif free_text in (m1_keywords):
            total_rating -= 1
        elif free_text in (p1_keywords):
            total_rating += 1
        elif free_text in (p2_keywords):
            total_rating += 2
    games.loc [game_index, "Your rating"] = total_rating / len (val)
    
    

/var/folders/tk/x_9q45v57_7cf8kzlygh1sfw0000gp/T/ipykernel_2123/165623461.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  games.loc [game_index, "Your rating"] = total_rating / len (val)


In [208]:
def grade_invented_game (games, invented_game): #No keywords
    #Metrics for time = 15:00
    l = games ["choices_at_15:00"].sum()
    choices_per_word_count = {w: games [games ["Word count"] == w]["choices_at_15:00"].sum()/l for w in games ["Word count"].unique()}
    choices_per_Game_section = {str (w): games [games ["Game section"] == w]["choices_at_15:00"].sum()/l for w in games ["Game section"].unique()} #ignore for now
    choices_per_num_of_sections = {w: games [games ["number of sections in description"] == w]["choices_at_15:00"].sum()/l for w in games ["number of sections in description"].unique()}
    choices_per_clip_len = {w: games [games ["Clip length [sec]"] == w]["choices_at_15:00"].sum()/l for w in games ["Clip length [sec]"].unique()}

    ##Let's match the new game
    #Weighted averages
    average_word_count = sum (np.array (list(choices_per_word_count.values())) * np.array (list(choices_per_word_count.keys())))
    average_number_of_sections = sum (np.array (list(choices_per_num_of_sections.values())) * np.array (list(choices_per_num_of_sections.keys())))
    average_clip_len = sum (np.array (list(choices_per_clip_len.values())) * np.array (list(choices_per_clip_len.keys())))

    #Keywords


    grade = 100 - abs (invented_game ["Clip length [sec]"] - average_clip_len)\
        - abs (invented_game ['number of sections in description'] - average_number_of_sections) * 3 - abs (invented_game ["Word count"] - average_word_count) / 5 \


    if grade > 100:
        grade = 100

    print (f"average_word_count: {average_word_count}\naverage_number_of_sections: {average_number_of_sections}\naverage_clip_len: {average_clip_len}")
    print (f"invented_game: {invented_game}")
    print (f"grade: {grade}")




In [243]:
def calculate_normalized_rating (games, invented_game):
    matching_rates = [] #Matching of "invented_game" to each of the other games
    for game_name, game in games.iterrows ():
        word_num_weight = abs (invented_game ["Word count"] - game ["Word count"]) / 150
        section_num_weight = abs (invented_game ["number of sections in description"] - game ["number of sections in description"]) / 20
        clip_length_weight = abs (invented_game ["Clip length [sec]"] - game ["Clip length [sec]"]) / 100

        matching = 1 - word_num_weight - section_num_weight - clip_length_weight
        matching_rates.append (matching)
    normalized_matching = np.array (matching_rates) / sum (matching_rates)
    normalized_rating = (normalized_matching * games ["Your rating"]).sum()
    return normalized_rating

In [271]:
other_peoples_ratings = pd.DataFrame ({"Setup": ["outside", "outside", "outside", "low parent energy", \
                                                 "low parent energy", "low parent energy", "low parent energy", \
                                                 "outside", "outside", "outside", "outside", "outside", "outside"], 
                                                 "Rating": [4, 4, 5, 3, 5, 5, 5, 3, 5, 1, 2, 3, 4]})

In [272]:
def grade_component_based_on_other_peoples_opinion (other_peoples_ratings):
    component = pd.DataFrame (columns= ["Setup", "Number of ratings", "Average rating"])
    for setup, val in other_peoples_ratings.groupby ("Setup"):
        component.loc [len (component), :] = [setup, len (val), np.mean(val ["Rating"])]

    return component

In [282]:
row

Setup                 outside
Number of ratings           9
Average rating       3.444444
Name: 1, dtype: object

In [286]:
def grade_invented_game_v2 (games, invented_game, other_peoples_ratings): #With keywords
    #Metrics for time = 15:00
    l = games ["choices_at_15:00"].sum()
    choices_per_word_count = {w: games [games ["Word count"] == w]["choices_at_15:00"].sum()/l for w in games ["Word count"].unique()}
    choices_per_Game_section = {str (w): games [games ["Game section"] == w]["choices_at_15:00"].sum()/l for w in games ["Game section"].unique()} #ignore for now
    choices_per_num_of_sections = {w: games [games ["number of sections in description"] == w]["choices_at_15:00"].sum()/l for w in games ["number of sections in description"].unique()}
    choices_per_clip_len = {w: games [games ["Clip length [sec]"] == w]["choices_at_15:00"].sum()/l for w in games ["Clip length [sec]"].unique()}

    ##Let's match the new game
    #Weighted averages
    average_word_count = sum (np.array (list(choices_per_word_count.values())) * np.array (list(choices_per_word_count.keys())))
    average_number_of_sections = sum (np.array (list(choices_per_num_of_sections.values())) * np.array (list(choices_per_num_of_sections.keys())))
    average_clip_len = sum (np.array (list(choices_per_clip_len.values())) * np.array (list(choices_per_clip_len.keys())))

    #Keywords
    normalized_rating_based_on_keywords = calculate_normalized_rating (games, invented_game)

    grade = 100 - abs (invented_game ["Clip length [sec]"] - average_clip_len)\
        - abs (invented_game ['number of sections in description'] - average_number_of_sections) * 3 - abs (invented_game ["Word count"] - average_word_count) / 5 \
        + normalized_rating_based_on_keywords * 30

    if grade > 100:
        grade = 100

    component = grade_component_based_on_other_peoples_opinion (other_peoples_ratings)


    print ("Your preferences:")
    print (f"number of words in the description: {average_word_count}\nnumber of sections in the description: {average_number_of_sections}\nfavorite clip length : {average_clip_len}")
    print ("----------")
    print (f"Based on your previous ratings, you will rate this game (on a scale from -2 to 2) as {normalized_rating_based_on_keywords:.2f}")
    print ("----------")
    # print (f"invented_game: {invented_game}")
    print (f"Matching level to you: {grade:.2f}%")
    print ("----------")
    print ("Other users' feedbacks:")
    for idx, row in component.iterrows ():
        x = row ["Setup"]
        y = row ["Number of ratings"]
        z = row ["Average rating"] 
        print (f"Level of matching to: {x}, based on {y} ratings: {z:.1f}")



In [287]:
invented_game = {'name': 'musical hot&cold',
 'Word count': 23,
 'Game section': 'Popular',
 'number of sections in description': 2,
 'Clip length [sec]': 20}

In [288]:
grade_invented_game_v2 (games, invented_game, other_peoples_ratings)

Your preferences:
number of words in the description: 21.0
number of sections in the description: 1.875
favorite clip length : 28.0
----------
Based on your previous ratings, you will rate this game (on a scale from -2 to 2) as -0.35
----------
Matching level to you: 80.68%
----------
Other users' feedbacks:
Level of matching to: low parent energy, based on 4 ratings: 4.5
Level of matching to: outside, based on 9 ratings: 3.4
